In [9]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
import os
import json 

# Initialize envirnonment variable OPENAI_API_KEY with your API key from the config.json file
with open('config.json', 'r') as file:
    config = json.load(file)
    os.environ['OPENAI_API_KEY'] = config.get('openaikey')


def has_subfolders(folder_path):
    # List all items (files and directories) in the specified folder
    items = os.listdir(folder_path)

    # Check if there are any subfolders
    for item in items:
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            return True  # At least one subfolder found

    return False  # No subfolders found


# Function to generate description for single code files using the OpenAI API
def generate_description(code):
    # Template for the prompt
    template = """Describe the purpose of this code: \n {code}
    Answer: """
    prompt = PromptTemplate(template=template,input_variables=['code'])        
    # OpenAI model
    model = OpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.5)       
    # LLMChain
    llm_chain = LLMChain(
    prompt=prompt,
    llm=model
    )
    # Run the model
    answer = llm_chain.run(code)
    return answer.strip()


# Function to generate description for a directory not containing subdirectories (only files) using the OpenAI API
def generate_only_files_dir_description(directory_path):

    files_descriptions = []

    # Template for the prompt
    template = """Describe the purpose or function of this directory in a codebase, considering the descriptions of the files 
    contained within this directory. Here are the descriptions of those files: \n {text}
    Answer: """
    prompt = PromptTemplate(template=template,input_variables=['text'])        

    # OpenAI model
    model = OpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.5)       

    # LLMChain
    llm_chain = LLMChain(
    prompt=prompt,
    llm=model
    )    

    # Loading all descriptions of the files inside the directory
    for filename in os.listdir(directory_path):
        if filename.endswith("_DESCRIPTION.txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as file:
                files_descriptions.append(file.read())

    # Concatenate the list of file descriptions into a single text string
    text_to_pass = "\n".join(files_descriptions)

    # Run the model
    answer = llm_chain.run(text_to_pass)

    # Extract the folder name from the directory path
    folder_name = os.path.basename(directory_path)

    # Write the directory description in a txt file with only the folder name
    with open(directory_path + "_folder_DESCRIPTION.txt", "w", encoding="utf-8") as txt_file:
        txt_file.write("#This is the description for directory " + directory_path + "\n")
        txt_file.write(str(answer.strip()))
    
    return answer.strip()

# Function to generate description for directory containing subdirectories (and files) using the OpenAI API
def generate_subdir_and_files_dir_description(directory_path):

    files_descriptions = []

    # Template for the prompt
    template = """Describe the purpose or function of this directory in a codebase, considering the descriptions of the files
    and of the folders contained within this directory. Here are the descriptions of them: \n {text}
    """
    prompt = PromptTemplate(template=template,input_variables=['text'])
    
    # OpenAI model
    model = OpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.2)
    
    # LLMChain
    llm_chain = LLMChain(
    prompt=prompt,
    llm=model
    )
    
    # Loading all descriptions of the files and folders inside the directory
    for filename in os.listdir(directory_path):
        if filename.endswith("_DESCRIPTION.txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as file:
                files_descriptions.append(file.read())

    # Concatenate the list of file descriptions into a single text string
    text_to_pass = "\n".join(files_descriptions)

    # Run the model
    answer = llm_chain.run(text_to_pass)

    # Extract the folder name from the directory path
    folder_name = os.path.basename(directory_path)

    # Write the directory description in a txt file with only the folder name
    with open(directory_path + "_folder_DESCRIPTION.txt", "w", encoding="utf-8") as txt_file:
        txt_file.write("#This is the description for directory " + directory_path + "\n")
        txt_file.write(str(answer.strip()))
    
    return answer.strip()

# Function to analyze code and generate descriptions for files and subfolders
def analyze_and_generate_descriptions(directory):

    folder_descriptions = {}

    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)

        if os.path.isfile(item_path) and not item_path.endswith("_DESCRIPTION.txt"):
            #print("file description already generated")
            # Read code from the file
            with open(item_path, 'r') as file:
                code = file.read()

            # Use OpenAI to generate a description based on the code
            file_description = generate_description(code) 
            
            # Write the file description in a txt file
            with open(os.path.splitext(item_path)[0] + "_DESCRIPTION.txt", "w", encoding="utf-8") as txt_file:
                txt_file.write("#This is the description for file " + os.path.splitext(item_path)[0] + "\n")
                txt_file.write(str(file_description))
        else:
            if os.path.isdir(item_path):
                # Generate descriptions for folders
                folder_descriptions[item] = analyze_and_generate_descriptions(item_path)

    if has_subfolders(directory): 
        folder_descriptions[directory] = generate_subdir_and_files_dir_description(directory)
    else:
        folder_descriptions[directory] = generate_only_files_dir_description(directory)
    
    return folder_descriptions


if __name__ == "__main__":
    # Specify the main directory of your source code repository
    main_directory = "CodeFiles\\antifrode-cut"

    # Generate descriptions for the main directory
    main_directory_description = analyze_and_generate_descriptions(main_directory)


file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
file description already generated
